# Deep Dream
## Introduction
In this notebook we are going to implement **Deep Dream** algorithm in **Keras**. This algorithm allows to generate art-like images from an initial image by adding activations filters through the process of gradient ascent of the trained model:

<img src='city.jpg' style='height:320px; width:550px'>

<img src='city_dream.png' style='height:320px; width:550px'>

Let's `import` convolutional base for our model pretrained on `imagenet`(weights='imagenet') dataset without fully-connected layers (include_top=False) and set learning phase to be zero (we are not going to train the model). In this case we are going to use **Inception v3** model rather than **Inception** of the first version how it was proposed in the [initial article](https://ai.googleblog.com/2015/07/deepdream-code-example-for-visualizing.html).

In [13]:
from keras.applications import inception_v3
from keras import backend as K
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

First of all we should choose layers with which we are going to make gradient ascents, thus let's look at the structure of the model and choose 'concatenation' ones. 

In [14]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
_______________________________________________________________________________________

Here I choosed first **ten `mixed` layers** to have deal with their filters but I am going to work just with the **five layers somewhere in the middle**(first 'concatenation' layers extract just simple geometrical features from the images but our goal is to derrive from the picture more complex ones and map it on our image). 

In [15]:
layers = {
#'mixed0': 0.8,
#'mixed1': 0.7,
'mixed2': 0.2,
'mixed3': 2.4,
'mixed4': 1.7,
'mixed5': 1.5,
'mixed6': 0.9,
#'mixed7': 0.9,
#'mixed8': 1.5,
#'mixed9': 2.9
}

Let's look at the output shape of choosed layers:

In [16]:
for layer in layers:
    print(model.get_layer(layer).output.shape.as_list())

[None, None, None, 288]
[None, None, None, 768]
[None, None, None, 768]
[None, None, None, 768]
[None, None, None, 768]


Then we should accumulate normalized `loss` value of the choosed layers. The slice of the `activation` we are going to use is of size $((h-4)\times(w-4)$ - `activation[:, 2: -2, 2: -2, :]`:

In [17]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer in layers:
    coeff = layers[layer]
    activation = model.get_layer(layer).output
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / K.prod(K.cast(K.shape(activation), 'float32'))

To **retrive gradients and losses** from the model we create a **function in Keras**:

In [18]:
dream_input = model.input
grads = K.gradients(loss, dream_input)[0]
grads = grads / K.maximum(K.mean(K.abs(grads)), 1e-10) # Normalize gradients
outputs = [loss, grads]
function = K.function([dream_input], outputs)

To print the value of loss we create evaluation function:

In [19]:
def evaluate(x):
    outs = function([x])
    loss = outs[0]
    grad = outs[1]
    return loss, grad

And eventually create the most important function to provide `gradient ascent`:

In [20]:
def gradient_ascent(image, iterations, lr, max_loss=None):
    for i in range(iterations):
        loss, grad = evaluate(image)
        if max_loss is not None and loss > max_loss:
            break
        print('iteration: {} ----- loss: {}'.format(i, loss))
        image += lr * grad
    return image

,where `lr` is equivalent to learning rate when we train a Neural Net, thus has approximately the same value ($\approx 5\cdot10^{-3}\div5\cdot10^{-4}$) and `max_loss` is hyperparameter that directly affects on how much we change the structure of choosed image ($\approx 10 \div 50$). 

**Let's also create functions to work with the image:**

We are going to work with one image but resized differently:

In [21]:
def resize(image, size):
    import numpy as np
    import scipy
    image = np.copy(image)
    factors = (1, float(size[0]) / image.shape[1],
               float(size[1]) / image.shape[2], 1)
    return scipy.ndimage.zoom(image, factors, order=1)

Then we `deprocess` the image to have pexel values from `zero` to `255`:

In [22]:
def deprocess(image):
    if K.image_data_format() == 'channels_first':
        image = image.reshape((3, image.shape[2], image.shape[3]))
        image = image.transpose((1, 2, 0))
    else:
        image = image.reshape((image.shape[1], image.shape[2], 3))
    image /= 2.
    image += 0.5
    image *= 255.
    image = np.clip(image, 0, 255).astype('uint8')
    return image

To save new image (maped with the grads):

In [23]:
def save(image, name):
    import imageio
    import numpy as np
    dep_image = deprocess(np.copy(image))
    imageio.imwrite(name, dep_image)

When we `preprocess` the `initial image` we expand its dimension (image is of shape **(height, width, 3)** but the `input` of the model is 4-dimensional tensor (*batch of images*) of shape **(num_images, height, width, 3)**):

In [24]:
def preprocess(images_path):
    from keras.preprocessing import image as img
    image = img.load_img(images_path)
    image = img.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = inception_v3.preprocess_input(image)
    return image

#### Here we load our initial image and choose the `number of octave` - number of generated images with different size and calculate their `shapes`: 

In [33]:
import numpy as np
num_octave = 3
octave_scale = 1.4
image = preprocess('paris.jpeg')
initial_shape = image.shape[1:3]
shapes = [initial_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in initial_shape])
    shapes.append(shape)
    
shapes = shapes[::-1]
initial_image = np.copy(image)
shrunk_initial_image = resize(image, shapes[0])

#### And eventually we run the process of `gradient ascent`.

### Note!
There is a problem when you reshape the `initial image` upside (from the image of small size ($S$) into big one ($B$)). During the process of resizing you lost some pixels (information about particular details). To prevent that or to add needed details we took into consideration the `initial image` of size $B$:
```python
image = gradient_ascent(image, iterations=..., lr=..., max_loss=...)
lost_details = initial_image_of_size_B - upscaled_shrunk_initial_of_size_B
image = image + lost_details
```

In [34]:
for shape in shapes:
    print('---------------------------------------')
    print('Processing image of shape: ', shape)
    print('---------------------------------------')
    image = resize(image, shape)
    image = gradient_ascent(image, iterations=60, lr=0.01, max_loss=40)
    upscaled_shrunk_initial = resize(shrunk_initial_image, shape)
    same_size_initial = resize(initial_image, shape)
    lost_detail = same_size_initial - upscaled_shrunk_initial
    image += lost_detail
    shrunk_initial_image = resize(initial_image, shape)
    save(image, name='dream_at_scale_' + str(shape) + '.png')
save(image, name='paris_dream.png')

---------------------------------------
Processing image of shape:  (551, 1224)
---------------------------------------
iteration: 0 ----- loss: 1.9878239631652832
iteration: 1 ----- loss: 2.5178980827331543
iteration: 2 ----- loss: 3.181368112564087
iteration: 3 ----- loss: 3.8393945693969727
iteration: 4 ----- loss: 4.466948509216309
iteration: 5 ----- loss: 5.073509216308594
iteration: 6 ----- loss: 5.658028602600098
iteration: 7 ----- loss: 6.235952377319336
iteration: 8 ----- loss: 6.793768882751465
iteration: 9 ----- loss: 7.353477954864502
iteration: 10 ----- loss: 7.895106315612793
iteration: 11 ----- loss: 8.410794258117676
iteration: 12 ----- loss: 8.932978630065918
iteration: 13 ----- loss: 9.41873836517334
iteration: 14 ----- loss: 9.916206359863281
iteration: 15 ----- loss: 10.4071044921875
iteration: 16 ----- loss: 10.85922908782959
iteration: 17 ----- loss: 11.323277473449707
iteration: 18 ----- loss: 11.761119842529297
iteration: 19 ----- loss: 12.209047317504883
iterat

C:\Users\79777\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


---------------------------------------
Processing image of shape:  (1080, 2400)
---------------------------------------
iteration: 0 ----- loss: 8.784514427185059
iteration: 1 ----- loss: 16.314661026000977
iteration: 2 ----- loss: 26.285091400146484


<img src='paris.jpeg' style='height:320px; width:630px'>

<img src='paris_dream.png' style='height:320px; width:630px'>

#### ! Now you can try it with different values of `hyperparameters` like `iterations, lr, max_loss`.